In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:

# change this to the folder where your CSVs live
BASE = "/content/drive/MyDrive/datasetnws/gtfs"

In [ ]:
!pip -q install folium

In [ ]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt

# Show more columns in outputs
pd.set_option("display.max_columns", 100)

# GTFS times can be >24:00:00; parse safely to seconds
def parse_gtfs_time(s):
    # returns seconds from midnight; NaN -> np.nan
    try:
        h, m, sec = map(int, str(s).split(":"))
        return h*3600 + m*60 + sec
    except Exception:
        return np.nan

# Route type names (GTFS)
ROUTE_TYPE = {
    0:"Tram/Light rail", 1:"Subway", 2:"Rail", 3:"Bus",
    4:"Ferry", 5:"Cable car", 6:"Gondola", 7:"Funicular",
    11:"Trolleybus", 12:"Monorail"
}


In [ ]:
agency   = pd.read_csv(f"{BASE}/agency.csv", dtype=str, low_memory=False)
routes   = pd.read_csv(f"{BASE}/routes.csv", dtype=str, low_memory=False)
trips    = pd.read_csv(f"{BASE}/trips.csv", dtype=str, low_memory=False)
stops    = pd.read_csv(f"{BASE}/stops.csv", dtype=str, low_memory=False)

# load only the columns we need from stop_times (it’s the huge one)
usecols = ["trip_id","arrival_time","departure_time","stop_id","stop_sequence"]
stop_times = pd.read_csv(f"{BASE}/stop_times.csv", usecols=usecols, dtype=str, low_memory=False)

calendar = pd.read_csv(f"{BASE}/calendar.csv", dtype=str, low_memory=False)
calendar_dates = pd.read_csv(f"{BASE}/calendar_dates.csv", dtype=str, low_memory=False, na_filter=False)


In [ ]:
for name, df in {
    "agency":agency, "routes":routes, "trips":trips, "stops":stops,
    "stop_times":stop_times, "calendar":calendar, "calendar_dates":calendar_dates
}.items():
    print(name, df.shape, "nulls:", df.isna().sum().sum())


In [ ]:
for name, df in [("trips", trips), ("stops", stops), ("agency", agency)]:
    print(f"\n{name.upper()}")
    print(df.isna().sum(), "\n")


In [ ]:
stops = stops.dropna(subset=["stop_lat","stop_lon"])


In [ ]:
# convert times to seconds for analysis
def time_to_sec(t):
    try:
        h,m,s = map(int, t.split(":"))
        return h*3600 + m*60 + s
    except:
        return np.nan

stop_times["arr_sec"] = stop_times["arrival_time"].apply(time_to_sec)
stop_times["dep_sec"] = stop_times["departure_time"].apply(time_to_sec)
stop_times["hour"]    = stop_times["dep_sec"] // 3600


In [ ]:
# drop rows with missing stop_ids or trip_ids
stop_times = stop_times.dropna(subset=["stop_id","trip_id"])

# drop trips without a route_id
trips = trips.dropna(subset=["route_id"])


In [ ]:
#Handle duplicates
print("Duplicates in stops:", stops.duplicated(subset=["stop_id"]).sum())
stops = stops.drop_duplicates(subset=["stop_id"])


In [ ]:
# service_ids running on Monday
svc_mon = calendar.loc[calendar["monday"]=="1", "service_id"].unique()
trips_mon = trips[trips["service_id"].isin(svc_mon)]
stop_times_mon = stop_times[stop_times["trip_id"].isin(trips_mon["trip_id"])]

print("Trips on Monday:", len(trips_mon), "Stop time rows:", len(stop_times_mon))


In [ ]:
master = (
    stop_times_mon
      .merge(trips,   on="trip_id",  how="left")
      .merge(routes,  on="route_id", how="left")
      .merge(stops,   on="stop_id",  how="left")
)

# Parse times to seconds to enable grouping by hour, etc.
master["arr_sec"] = master["arrival_time"].map(parse_gtfs_time)
master["dep_sec"] = master["departure_time"].map(parse_gtfs_time)
master["hour"]    = (master["arr_sec"] // 3600).astype("Int64")

# Map route_type to readable labels if present
if "route_type" in master.columns:
    master["route_type_name"] = master["route_type"].astype(float).astype(int, errors="ignore").map(ROUTE_TYPE)
master.head()


**ANALYSIS A — Busiest stops (by scheduled service)**

In [ ]:
busiest_stops = (master.groupby(["stop_id","stop_name"])["trip_id"]
                        .nunique()
                        .sort_values(ascending=False)
                        .head(20)
                        .reset_index(name="unique_trips"))
busiest_stops


In [ ]:
plt.figure(figsize=(10,5))
plt.barh(busiest_stops["stop_name"].iloc[::-1], busiest_stops["unique_trips"].iloc[::-1])
plt.title("Top 20 Stops by Unique Trips (proxy for usage)")
plt.xlabel("Unique Trips (per day)")
plt.tight_layout()
plt.show()


**ANALYSIS B — Busiest routes**

In [ ]:
routes_by_trips = (master.groupby(["route_id","route_short_name","route_long_name"])["trip_id"]
                          .nunique()
                          .sort_values(ascending=False)
                          .head(20)
                          .reset_index(name="unique_trips"))
routes_by_trips


**ANALYSIS C — Peak‑hour profile**

In [ ]:
hourly = master.dropna(subset=["hour"]).groupby("hour")["trip_id"].count().reset_index(name="stop_events")
plt.figure(figsize=(10,4))
plt.bar(hourly["hour"], hourly["stop_events"])
plt.xticks(range(0, 25))
plt.title("Stop Events by Hour (all routes)")
plt.xlabel("Hour of day")
plt.ylabel("Stop events")
plt.show()


**ANALYSIS D — Service by day of week**

In [ ]:
days = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
day_counts = calendar[days].astype(int).sum()
plt.figure(figsize=(8,4))
plt.bar(day_counts.index.str.title(), day_counts.values)
plt.title("Service Availability by Day (count of service_ids running)")
plt.xticks(rotation=0)
plt.show()


**ANALYSIS E — Headway (average scheduled wait) per route & hour**

In [ ]:
# choose departure seconds for stability
evt = master.dropna(subset=["dep_sec","hour"]).copy()
evt["dep_sec"] = evt["dep_sec"].astype(int)

# per (route_id, hour) sort by time and compute diffs
def avg_headway(group):
    g = group.sort_values("dep_sec")["dep_sec"].values
    if len(g) < 2:
        return np.nan
    gaps = np.diff(g)
    return np.mean(gaps)/60.0  # minutes

headway = (evt.groupby(["route_id","hour"])
              .apply(avg_headway)
              .reset_index(name="avg_headway_min")
              .dropna())
headway.sort_values(["avg_headway_min"]).head(20)


**Public Transport Efficiency Analysis (GTFS Data)**

**Busiest Stops**

In [ ]:
busiest_stops = (
    master.groupby(["stop_id","stop_name"])["trip_id"]
          .nunique()
          .sort_values(ascending=False)
          .head(20)
          .reset_index(name="unique_trips")
)

plt.figure(figsize=(10,6))
plt.barh(busiest_stops["stop_name"][::-1], busiest_stops["unique_trips"][::-1])
plt.title("Top 20 Busiest Stops by Unique Trips")
plt.xlabel("Unique Trips")
plt.ylabel("Stop Name")
plt.tight_layout()
plt.show()


**Busiest Routes**

In [ ]:
label_col = "route_short_name" if "route_short_name" in master.columns else "route_id"
busiest_routes = (
    master.groupby(["route_id", label_col])["trip_id"]
          .nunique()
          .sort_values(ascending=False)
          .head(15)
          .reset_index(name="unique_trips")
)

plt.figure(figsize=(12,6))
x = np.arange(len(busiest_routes))
plt.bar(x, busiest_routes["unique_trips"].values)
plt.xticks(x, busiest_routes[label_col].astype(str).values, rotation=45, ha="right")
plt.title("Top 15 Routes by Number of Trips")
plt.ylabel("Unique Trips")
plt.tight_layout()
plt.show()


**Peak Hours**

In [ ]:
hourly = (
    master[master["hour"].notna()]
      .groupby(master["hour"].astype(int))["trip_id"]
      .count()
      .reset_index(name="stop_events")
      .sort_values("hour")
)

plt.figure(figsize=(11,5))
plt.plot(hourly["hour"], hourly["stop_events"], marker="o")
plt.xticks(range(0,24))
plt.title("Stop Events by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Stop Events (scheduled)")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


**Weekly Service**

In [ ]:
days = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
day_counts = calendar[days].astype(int).sum()

plt.figure(figsize=(9,5))
plt.bar([d.title() for d in days], day_counts.values)
plt.title("Service Availability by Day of Week")
plt.ylabel("Count of Service IDs Running")
plt.tight_layout()
plt.show()


**Headway (average scheduled wait)**

In [ ]:
evt = master[master["dep_sec"].notna() & master["hour"].notna()].copy()
evt["dep_sec"] = evt["dep_sec"].astype(int)
evt["hour"]    = evt["hour"].astype(int)

# average headway (minutes) for each (route_id, hour)
def avg_headway(group):
    sec = np.sort(group["dep_sec"].values)
    if sec.size < 2:
        return np.nan
    gaps = np.diff(sec)
    return gaps.mean() / 60.0

headway = (
    evt.groupby(["route_id","hour"])
       .apply(avg_headway)
       .reset_index(name="avg_headway_min")
       .dropna()
)

# Boxplot by hour
hour_groups = [headway.loc[headway["hour"]==h, "avg_headway_min"].values
               for h in range(24)]
plt.figure(figsize=(12,5))
plt.boxplot(hour_groups, positions=list(range(24)), showfliers=False)
plt.title("Average Headway by Hour (minutes)")
plt.xlabel("Hour of Day")
plt.ylabel("Average Headway (min)")
plt.ylim(0, 60)  # clamp if outliers are huge
plt.tight_layout()
plt.show()

# Histogram of all headways (clip extreme outliers for readability)
vals = np.clip(headway["avg_headway_min"].values, 0, 90)
plt.figure(figsize=(9,5))
plt.hist(vals, bins=30)
plt.title("Distribution of Average Headways (minutes)")
plt.xlabel("Average Headway (min)")
plt.ylabel("Route-Hour counts")
plt.tight_layout()
plt.show()


**Highlight busiest stops on the same scatter**

In [ ]:
# Base map (Sydney CBD center; change if using Melbourne)
m = folium.Map(location=[-33.8688, 151.2093], zoom_start=11)

# Plot ALL stops faintly
for _, r in stops.iterrows():
    if pd.notna(r.get("stop_lat")) and pd.notna(r.get("stop_lon")):
        folium.CircleMarker([float(r.stop_lat), float(r.stop_lon)],
                            radius=1, opacity=0.5, fill=True, fill_opacity=0.5).add_to(m)

# Highlight top 200 busiest stops in red (by unique trips)
topN = busiest_stops.head(200).merge(stops[["stop_id","stop_lat","stop_lon"]], on="stop_id")
for _, r in topN.iterrows():
    folium.CircleMarker([float(r.stop_lat), float(r.stop_lon)],
                        radius=4, popup=f"{r.stop_name} ({r.unique_trips} trips)",
                        color="red", fill=True).add_to(m)

m  # displays in Colab


In [ ]:
busiest_stops.to_csv(f"{BASE}/busiest_stops.csv", index=False)
routes_by_trips.to_csv(f"{BASE}/busiest_routes.csv", index=False)
hourly.to_csv(f"{BASE}/hourly_stop_events.csv", index=False)
headway.to_csv(f"{BASE}/route_hour_headway.csv", index=False)
